<a href="https://colab.research.google.com/github/zhenghaojiang/rl_dsge/blob/main/PF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import math
import matplotlib.pyplot as plt
from numpy.random import normal,uniform
from scipy.stats import t,gamma,norm
from abc import ABC, abstractmethod
from sympy import Symbol,solve

In [ ]:
class Solver(ABC):

    def __init__(self,params,struc_params):
        # Constants
        self.r = params["r"]
        self.delta = params["delta"]
        # Structural Params
        self.theta = struc_params[0]
        self.rho = struc_params[1]
        self.sigma = struc_params[2]
    
    def pi(self,k,z):
        return z*k**self.theta

    def psi(self,I,k):
        return 0.01*(I**2)/(2*k)

    def modeldefs(self,k,kp):
        I = kp-(1-self.delta)*k
        return I

    @abstractmethod
    def dsge_solve(self):
        pass

In [ ]:
class Linear(Solver):

    def __init__(self,params,struc_params):
        super(Linear, self).__init__(params,struc_params)
        # Constants
        self.k_st = (((self.r+self.delta)*(1+0.01*self.delta)-0.005*(self.delta**2))/self.theta) ** (1/(self.theta-1))
        self.I_st = self.delta*self.k_st
        self.m1 = self.theta*(self.theta-1)*(self.k_st**(self.theta-2))-0.01*(self.I_st**2)/(self.k_st**3) \
                  -0.01*(1-self.delta)*self.I_st/(self.k_st**2)
        self.m2 = 0.01*(self.I_st/(self.k_st**2)+(1-self.delta)/self.k_st)

    def dsge_solve(self):
        A = Symbol('A')
        B = Symbol('B')
        solved_value = solve([0.01*(1+self.r)*(A/self.k_st-self.I_st/(self.k_st**2))-(1-self.delta+A)*(self.m1+self.m2*A), 
                              0.01*(1+self.r)*B/self.k_st-self.theta*(self.k_st**(self.theta-1))*self.rho \
                              -self.m1*B-self.m2*B*(A+self.rho)], 
                             [A, B])
        kab = np.array(solved_value)[0]
        return kab

In [ ]:
np.random.seed(1111)

basic_params = {'r': 1/24,
                'delta': 0.15}

basic_struc_params = [0.70,0.70,0.15]

struc_params_init = [0.50,0.50,0.30]

def ktrans(k,I,delta):
    return (1-delta) * k + I

def ztrans(z,rho,sigma):
    lnz = np.log(z)
    lnz_new = rho * lnz + normal(0,sigma)
    return np.exp(lnz_new)

def action(k,z,kab,k_st = 74.79,I_st = 11.22):
  I_tilt = kab[0]*(k-k_st)+kab[1]*(z-1)
  return I_tilt+I_st

def sim_k(params: dict = basic_params, struc_params: np.array = basic_struc_params, kab: np.array = None, T = 500): 
    r = params["r"]
    delta = params["delta"]
    theta,rho,sigma = struc_params

    z0 = 1.0
    k0 = 70.0
    I0 = action(k0,z0,kab)
    
    k_sim = []
    I_sim = []
    z_sim = []
    for i in range(T):
        if i==0:
            z_sim.append(z0)
            k_sim.append(k0)
            I_sim.append(I0)
        else:
            z_sim.append(ztrans(z_sim[i-1],rho,sigma))
            k_sim.append(ktrans(k_sim[i-1],I_sim[i-1],delta))
            I_sim.append(action(k_sim[i],z_sim[i],kab))
    return k_sim

solver = Linear(params=basic_params,struc_params=basic_struc_params)
kab = solver.dsge_solve()
k_real = sim_k(kab=kab)

def prior(struc_params: np.array = None): 
    theta,rho,sigma = struc_params
    p_theta = gamma.pdf(theta,2,scale=theta/2)
    p_rho = gamma.pdf(rho,2,scale=rho/2)
    p_sigma = gamma.pdf(sigma,2,scale=sigma/2)
    return p_theta * p_rho * p_sigma

def loglikelihoodfunc(params: dict = basic_params, struc_params: np.array = None, N = 1000, T = 500, k_real = k_real, kab: np.array = None):
    r = params["r"]
    delta = params["delta"]
    theta,rho,sigma = struc_params

    loglikeli = 0.0
    k = np.ones(N) * 70.0
    z = np.ones(N)
    for i in range(T):
        z = ztrans(z,rho,sigma)
        I = action(k,z,kab)
        k_cond = ktrans(k,I,delta)
        k_obs = k_real[i] 
        v = np.array(k_obs - k_cond,dtype=float)
        q = norm.pdf(v,scale=20)/sum(norm.pdf(v,scale=20))
        k = np.random.choice(k_cond,N,replace=True,p=q)
        loglikeli = loglikeli + math.log(np.mean(norm.pdf(v,scale=20)))
    return loglikeli

def mcmc(M = 20, struc_params_init: dict = struc_params_init):

    def threshold(params_new: np.array, params_old: np.array):
        prior_mol = prior(params_new)
        loglikeli_mol = loglikelihoodfunc(struc_params=params_new, kab=kab)
        prior_den = prior(params_old)
        loglikeli_den = loglikelihoodfunc(struc_params=params_old, kab=kab)
        log_res = np.log(prior_mol) + loglikeli_mol - np.log(prior_den) - loglikeli_den
        if log_res > 200:
            res = math.inf
        else:
            res = np.exp(log_res)
        return res
    
    params = struc_params_init
    # params_vec = []
    for _ in range(M):
        params_p = [max(params[0] + normal(scale=0.05),0.001),
                    max(params[1] + normal(scale=0.05),0.001),
                    max(params[2] + normal(scale=0.03),0.001)]
        chi = uniform()
        if chi <= threshold(params_p,params):
            params_up = params_p
        else:
            params_up = params
        params = params_up
        print(params)

In [ ]:
mcmc()